# Address Extraction and Masking using NER (Named Entity Recognition) BERT Variants

In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline


model_name = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)


ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, device=0)

/Users/rayaneghilene/Documents/Ollama/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/rayaneghilene/Documents/Ollama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if yo

### We pass the following arguments for address extraction

B-LOC	Beginning of a location right after another location \
I-LOC	Location

See https://huggingface.co/dslim/bert-base-NER for more details

In [2]:
text = "Send the package to 123 Main St, Springfield, IL 62704. " \
       "Or alternatively, you can send it to 456 Elm St, Shelbyville, IL 62565."\
       "you cna also send it to 75 Rue de la Tunisie, Nanterre, France"

ner_results = ner_pipeline(text)

address_tokens = [res['word'] for res in ner_results if res['entity'] == 'B-LOC' or res['entity'] == 'I-LOC']
address = ' '.join(address_tokens)

print("Extracted Address:", address)

ner_results = ner_pipeline(text)
address_tokens = [res['word'] for res in ner_results if res['entity'] == 'B-LOC' or res['entity'] == 'I-LOC']
print ("Address Tokens are ", address_tokens)

Extracted Address: Main St Springfield IL Elm St Shelby ##ville IL Rue de la Tu ##nis ##ie Nan ##ter ##re France
Address Tokens are  ['Main', 'St', 'Springfield', 'IL', 'Elm', 'St', 'Shelby', '##ville', 'IL', 'Rue', 'de', 'la', 'Tu', '##nis', '##ie', 'Nan', '##ter', '##re', 'France']


### Token Masking

In [3]:
import re
def mask_words(text, words_to_mask):
  for word in words_to_mask:
    text = re.sub(r'\b{}\b'.format(re.escape(word)), '[MASK]', text)
  return text

masked_text = mask_words(text, address_tokens)
print(masked_text)

Send the package to 123 [MASK] [MASK], [MASK], [MASK] 62704. Or alternatively, you can send it to 456 [MASK] [MASK], Shelbyville, [MASK] 62565.you cna also send it to 75 [MASK] [MASK] [MASK] Tunisie, Nanterre, [MASK]


# \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

## You can use a different NER model

Here we use a Distilled version of BERT, fine tuned for NER \
Model card: https://huggingface.co/dslim/distilbert-NER

In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline


model_name = "dslim/distilbert-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)


ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, device=0)

text = "Send the package to 123 Main St, Springfield, IL 62704. " \
       "Or alternatively, you can send it to 456 Elm St, Shelbyville, IL 62565."\
       "you cna also send it to 75 Rue de la Tunisie, Nanterre, France"

ner_results = ner_pipeline(text)
address_tokens = [res['word'] for res in ner_results if res['entity'] == 'B-LOC' or res['entity'] == 'I-LOC']
print ("Address Tokens are ", address_tokens)
masked_text = mask_words(text, address_tokens)
print(masked_text)

Address Tokens are  ['Main', 'St', 'Springfield', 'IL', 'Elm', 'St', 'Shelby', '##ville', 'IL', 'Rue', 'de', 'la', 'Tu', '##nis', '##ie', 'Nan', '##ter', '##re', 'France']
Send the package to 123 [MASK] [MASK], [MASK], [MASK] 62704. Or alternatively, you can send it to 456 [MASK] [MASK], Shelbyville, [MASK] 62565.you cna also send it to 75 [MASK] [MASK] [MASK] Tunisie, Nanterre, [MASK]
